# Yellow Taxi Data - Schema Analysis

In [1]:
# Define the base URL pattern
base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{}.parquet"

# Define date range
start_date = "2009-01"
end_date = "2024-08"

In [2]:
! pip3 install duckdb

In [3]:
import duckdb

con = duckdb.connect()
con.install_extension("httpfs")
con.load_extension("httpfs")

In [4]:
import pandas as pd
import duckdb
import time
import logging

# Create a DuckDB connection
con = duckdb.connect()

# date range
dates = pd.date_range(start_date, end_date, freq="MS").strftime("%Y-%m").tolist()

# Generate the list of SQL commands
sql_commands = [
    f"SELECT * FROM parquet_schema('{base_url.format(date.replace('-', '-'))}')" for date in dates
]

def load_with_backoff(con, sql_command, max_retries=10, initial_delay=15, backoff_factor=2):
    """
    Load data using a SQL command with retry and backoff logic.

    Args:
        con (duckdb.DuckDBPyConnection): DuckDB connection object.
        sql_command (str): The SQL command to execute.
        max_retries (int): Maximum number of retries.
        initial_delay (int): Initial delay in seconds.
        backoff_factor (int): Factor by which the delay increases after each retry.

    Returns:
        pd.DataFrame: DataFrame containing the loaded data, or None if it failed.
    """
    delay = initial_delay
    for attempt in range(max_retries):
        try:
            print(f"Executing SQL: {sql_command} (Attempt {attempt + 1})...")
            return con.sql(sql_command).df()
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Failed to execute SQL, retrying in {delay} seconds... ({attempt + 1}/{max_retries})")
                time.sleep(delay)
                delay *= backoff_factor
            else:
                logging.error(f"Failed to execute SQL after {max_retries} retries: {e}")
    return None


# Load all SQL commands
dfs = [load_with_backoff(con, sql) for sql in sql_commands]

# Filter out None results
dfs = [df for df in dfs if df is not None]

# Concatenate all DataFrames into one
if dfs:
    final_df = pd.concat(dfs, ignore_index=True)
    print(f"Loaded {len(final_df)} rows from {len(dfs)} files.")
else:
    print("No data loaded.")

# Save to a single Parquet file (optional)
final_df.to_parquet("combined_tripdata_200901_to_202408.parquet")


Executing SQL: SELECT * FROM parquet_schema('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet') (Attempt 1)...
Failed to execute SQL, retrying in 15 seconds... (1/10)
Executing SQL: SELECT * FROM parquet_schema('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet') (Attempt 2)...
Failed to execute SQL, retrying in 30 seconds... (2/10)
Executing SQL: SELECT * FROM parquet_schema('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet') (Attempt 3)...
Failed to execute SQL, retrying in 60 seconds... (3/10)
Executing SQL: SELECT * FROM parquet_schema('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet') (Attempt 4)...
Executing SQL: SELECT * FROM parquet_schema('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-02.parquet') (Attempt 1)...
Executing SQL: SELECT * FROM parquet_schema('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-03.parqu

In [5]:
pd.set_option('display.max_colwidth', None)

final_df

,file_name,name,type,type_length,repetition_type,num_children,converted_type,scale,precision,field_id,logical_type
0,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,schema,None,None,REQUIRED,18.0,None,NaN,NaN,NaN,None
1,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,vendor_name,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType()
2,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Trip_Pickup_DateTime,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType()
3,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Trip_Dropoff_DateTime,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType()
4,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Passenger_Count,INT64,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
3733,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,tolls_amount,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None
3734,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,improvement_surcharge,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None
3735,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,total_amount,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None
3736,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,congestion_surcharge,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None


In [6]:
def add_yyyy_mm_column(df):
  """Adds a new column 'yyyy_mm' to the DataFrame based on filename patterns"""

  df['yyyy_mm'] = df['file_name'].str.extract(r'(\d{4}-\d{2})')
  return df


# Apply the function to your DataFrame
final_df = add_yyyy_mm_column(final_df.copy())  # Operate on a copy to avoid modifying original data

final_df

,file_name,name,type,type_length,repetition_type,num_children,converted_type,scale,precision,field_id,logical_type,yyyy_mm
0,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,schema,None,None,REQUIRED,18.0,None,NaN,NaN,NaN,None,2009-01
1,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,vendor_name,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType(),2009-01
2,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Trip_Pickup_DateTime,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType(),2009-01
3,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Trip_Dropoff_DateTime,BYTE_ARRAY,None,OPTIONAL,NaN,UTF8,NaN,NaN,NaN,StringType(),2009-01
4,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2009-01.parquet,Passenger_Count,INT64,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None,2009-01
...,...,...,...,...,...,...,...,...,...,...,...,...
3733,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,tolls_amount,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None,2024-08
3734,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,improvement_surcharge,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None,2024-08
3735,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,total_amount,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None,2024-08
3736,https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet,congestion_surcharge,DOUBLE,None,OPTIONAL,NaN,None,NaN,NaN,NaN,None,2024-08


Assume col names will be lowercased in VastDB

In [7]:
final_df['name'] = final_df['name'].str.lower()

Create the summary report

In [8]:
schemas = (
    final_df.groupby("yyyy_mm", group_keys=False)
    .apply(lambda df: df[["name", "type"]].set_index("name").to_dict()["type"], include_groups=False)
)

schemas = schemas.to_dict()

schema_changes = []
previous_schema = None

# Get the initial schema
initial_schema = schemas[min(schemas.keys())]

# Add a row for the initial schema
schema_changes.insert(0, {
    "file_name": "Initial Schema",
    "added_columns": [col for col in initial_schema.keys() if col != 'schema'],
    "removed_columns": [],
    "changed_columns": {}
})

for file_name, schema in schemas.items():
    if previous_schema is not None:
        added = set(schema.keys()) - set(previous_schema.keys())
        removed = set(previous_schema.keys()) - set(schema.keys())
        changed = {
            col: (previous_schema[col], schema[col])
            for col in set(previous_schema.keys()) & set(schema.keys())
            if previous_schema[col] != schema[col]
        }

        schema_changes.append({
            "file_name": file_name,
            "added_columns": list(added),
            "removed_columns": list(removed),
            "changed_columns": changed,
        })
    previous_schema = schema

changes_df = pd.DataFrame(schema_changes)

pd.set_option("display.max_rows", None)

print("Schema Drift Summary:")
changes_df

Schema Drift Summary:


,file_name,added_columns,removed_columns,changed_columns
0,Initial Schema,"[vendor_name, trip_pickup_datetime, trip_dropoff_datetime, passenger_count, trip_distance, start_lon, start_lat, rate_code, store_and_forward, end_lon, end_lat, payment_type, fare_amt, surcharge, mta_tax, tip_amt, tolls_amt, total_amt]",[],{}
1,2009-02,[],[],{}
2,2009-03,[],[],{}
3,2009-04,[],[],{}
4,2009-05,[],[],{}
5,2009-06,[],[],{}
6,2009-07,[],[],{}
7,2009-08,[],[],{}
8,2009-09,[],[],{}
9,2009-10,[],[],{}


Latest Schema

In [12]:
latest_schema = schemas[max(schemas.keys())]
del latest_schema['schema']
latest_schema

{'vendorid': 'INT32',
 'tpep_pickup_datetime': 'INT64',
 'tpep_dropoff_datetime': 'INT64',
 'passenger_count': 'INT64',
 'trip_distance': 'DOUBLE',
 'ratecodeid': 'INT64',
 'store_and_fwd_flag': 'BYTE_ARRAY',
 'pulocationid': 'INT32',
 'dolocationid': 'INT32',
 'payment_type': 'INT64',
 'fare_amount': 'DOUBLE',
 'extra': 'DOUBLE',
 'mta_tax': 'DOUBLE',
 'tip_amount': 'DOUBLE',
 'tolls_amount': 'DOUBLE',
 'improvement_surcharge': 'DOUBLE',
 'total_amount': 'DOUBLE',
 'congestion_surcharge': 'DOUBLE',
 'airport_fee': 'DOUBLE'}